In [1]:
#!pip install twitterscraper
# !pip install lxml

In [3]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm
import seaborn as sns
import pymysql
import requests
from bs4 import BeautifulSoup
import datetime

import json

import re


mpl.rcParams['axes.unicode_minus'] = False
path = 'C:/Windows/Fonts/malgun.ttf'
font_name = fm.FontProperties(fname=path, size=50).get_name()
plt.rc('font', family=font_name)

plt.rcParams['figure.figsize'] = (15,5)
plt.rcParams['font.size'] = 14

In [4]:
url = 'https://sports.news.naver.com/kbaseball/news/list.nhn?date=20200801&isphoto=N&type=team&team=NC'
res = requests.get(url).json()
team_news_list = res['list']
# bs = BeautifulSoup(res.text,'html.parser')

In [5]:
# def parse_content(url):
def parse_content(url):
    res = requests.get(url)
    bs = BeautifulSoup(res.text,'html.parser')
    content = bs.select('#newsEndContents')[0].text.strip()
    content = content.replace('/','')
    content = re.sub('\[(.*?)\]', '', content)
    content = re.sub('[a-zA-Z0-9]+@[a-zA-Z0-9]+([\.co\.kr]|[\.com])+', '', content)
    content = re.sub('[현]+(.*?)[다.]+', '', content)
    return content

In [6]:
# https://sports.news.naver.com/news.nhn?oid=425&aid=0000109083
news_df = []
for news in team_news_list:
    news_dic = {}
    content_url = 'https://sports.news.naver.com/news.nhn?oid={}&aid={}'.format(news['oid'],news['aid'])
    news_dic['title'] = news['title']
    news_dic['date'] = news['datetime']
    news_dic['content'] = parse_content(content_url)
    news_df.append(news_dic)
#     news_df.append(pd.DataFrame(data=news_dic))

In [7]:
news_df

[{'title': 'NC 3루 코치는 왜 이상호를 막았을까?...승부 가른 NC의 주루 미스',
  'date': '2020.08.01 23:12',
  'content': '나성범에게 한 방을 기대했다가 승리를 놓친 NC 다이노스 1일 창원 NC 파크에서 열린 ‘2020 신한은행 SOL KBO 리그’ NC 다이노스 대 두산 베어스 경기.8회 말 NC가 6-8로 뒤진 2사 상황에서 대타 박민우는 좌측 2루타를 쳤다. 그 후 박민우는 대주자 이상호로 교체됐다. 이어 이명기는 좌측 안타를 쳤다. 그리 빠른 타구도 아니었다. 그런데 2루 주자 이상호는 홈으로 들어오지 않고 3루에서 멈췄다. 3루 코치가 두 팔을 들어 이상호의 홈 질주를 막은 것이다. 홈으로 갔어도 승부가 될 수 있었다. 게다가 두산 좌익수는 어깨가 그리 좋지 않은 김재환이었다. 이상호가 홈을 밟았다면 NC는 한 점 차로 따라 붙을 수 있었다.  NC 3루 코치는 도대체 왜 이상호를 막았을까?TV 해설자는 다음 타자가 나성범이었기 때문일 수 있다고 추측했다. 나성범에게 3점 홈런을 기대했다는 것이다. 경기 막판, 그것도 9회를 남기고 있는 상황에서의 1점 차와 2점 차는 확연히 다르다. 1점 차 상황에서 상대 마무리 투수가 받는 압박감도 그렇지만, 공격하는 입장에서는 안타가 아니고 상대 에러만 나와도 1점을 얻을 수 있게 된다. 하나 물어보자.2루타를 치고 나간 박민우 대신 이상호를 대주자로 내보낸 이유가 무엇인가?발 빠른 이상호가 안타 하나로 홈을 밟을 수 있도록 하기 위해 그런 게 아니었던가?그렇다면, 이명기의 안타 때 이상호를 홈으로 보냈어야 했다.   상황적으로 봤을 때 이상호는 홈에서 좋은 승부를 벌일 수 있었다.설령 홈에서 아웃이 된다 해도, 9회 말을 나성범부터 시작할 수 있었다.이상호가 홈에 들어오지 못하자 나성범은 자신이 3점 홈런을 쳐야 한다는 부담감을 가질 수밖에 없었다. 타석에 들어서는 마음가짐과 타격 자세부터 달라진다. 결국 나성범은 큼지막한 외야 플라이로 아웃돼 NC는 더 

In [36]:
for i in range(len(news_df)):
    news_df[i]['content'] = news_df[i]['content'].split('\n')[0].split('▶')[0].split('\t')[0]
    for w in ['무단전재 및 재배포 금지', '재배포 금지', '재배포금지', 'ⓒGettyimages']:
        if w in news_df[i]['content']:
            content = content.replace(w, '')

In [37]:
news_df

[{'title': 'NC 3루 코치는 왜 이상호를 막았을까?...승부 가른 NC의 주루 미스',
  'date': '2020.08.01 23:12',
  'content': '나성범에게 한 방을 기대했다가 승리를 놓친 NC 다이노스 1일 창원 NC 파크에서 열린 ‘2020 신한은행 SOL KBO 리그’ NC 다이노스 대 두산 베어스 경기.8회 말 NC가 6-8로 뒤진 2사 상황에서 대타 박민우는 좌측 2루타를 쳤다. 그 후 박민우는 대주자 이상호로 교체됐다. 이어 이명기는 좌측 안타를 쳤다. 그리 빠른 타구도 아니었다. 그런데 2루 주자 이상호는 홈으로 들어오지 않고 3루에서 멈췄다. 3루 코치가 두 팔을 들어 이상호의 홈 질주를 막은 것이다. 홈으로 갔어도 승부가 될 수 있었다. 게다가 두산 좌익수는 어깨가 그리 좋지 않은 김재환이었다. 이상호가 홈을 밟았다면 NC는 한 점 차로 따라 붙을 수 있었다.  NC 3루 코치는 도대체 왜 이상호를 막았을까?TV 해설자는 다음 타자가 나성범이었기 때문일 수 있다고 추측했다. 나성범에게 3점 홈런을 기대했다는 것이다. 경기 막판, 그것도 9회를 남기고 있는 상황에서의 1점 차와 2점 차는 확연히 다르다. 1점 차 상황에서 상대 마무리 투수가 받는 압박감도 그렇지만, 공격하는 입장에서는 안타가 아니고 상대 에러만 나와도 1점을 얻을 수 있게 된다. 하나 물어보자.2루타를 치고 나간 박민우 대신 이상호를 대주자로 내보낸 이유가 무엇인가?발 빠른 이상호가 안타 하나로 홈을 밟을 수 있도록 하기 위해 그런 게 아니었던가?그렇다면, 이명기의 안타 때 이상호를 홈으로 보냈어야 했다.   상황적으로 봤을 때 이상호는 홈에서 좋은 승부를 벌일 수 있었다.설령 홈에서 아웃이 된다 해도, 9회 말을 나성범부터 시작할 수 있었다.이상호가 홈에 들어오지 못하자 나성범은 자신이 3점 홈런을 쳐야 한다는 부담감을 가질 수밖에 없었다. 타석에 들어서는 마음가짐과 타격 자세부터 달라진다. 결국 나성범은 큼지막한 외야 플라이로 아웃돼 NC는 더 

In [17]:
p = re.compile(r'\[(.*?)\]$')
e = re.compile(r'^/?\s?[a-zA-Z0-9]+@[a-zA-Z]+([\.co\.kr]|[\.com])+')

In [34]:
url = 'https://sports.news.naver.com/news.nhn?oid=109&aid=0004250320'
res = requests.get(url)
bs = BeautifulSoup(res.text,'html.parser')
content = bs.select('#newsEndContents')[0].text.strip()
content = content.replace('/','')
content = re.sub('\[(.*?)\]', '', content)
content = re.sub('[a-zA-Z0-9]+@[a-zA-Z0-9]+([\.co\.kr]|[\.com])+', '', content)
content = re.sub('[현]+(.*?)[다.]+', '', content)
content = content.split('\n')[0].split('▶')[0].split('\t')[0]
for i in ['무단전재 및 재배포 금지', '재배포 금지', '재배포금지', 'ⓒGettyimages']:
    if i in content:
        content = content.replace(i, '')
print(content)
# return content

 ‘최애 트로트 그룹’의 첫 공연에 예상치 못한 관객들이 초대돼 귀추가 주목된다. 오늘(1일) 오후 9시 50분 방송되는 MBC 예능프로그램 ‘최애 엔터테인먼트’(연출 오누리, 이민지) 5회에서는 ‘최애’ 멤버들이 우여곡절 끝에 준비한 첫 공연을 시작, 갈고 닦은 실력을 뽐낸다. 이날 방송에서는 ‘최애’ 멤버들의 공연 장소에 도착, 위압감 느껴지는 야외무대에 일동 긴장하는 모습을 보인다. MJ와 추혁진은 “안전한 곳 맞죠?”, “약간 무서운데”라며 불안한 눈빛을 보이는가 하면, 옥진욱은 “이런 걸 또 언제 해보겠어”라며 막내의 긍정적인 에너지를 발산하기도. 과연 이들을 이토록 떨게 한 무대는 어떤 모습일지 궁금증을 더하고 있다. 또한 첫 무대에 오른 ‘최애’ 멤버들이 오색찬란한 트로트 가수 의상을 맞춰 입어 눈길을 끈다. 살짝만 움직여도 빤짝이는 셔츠핏과 부쩍 물오른 외모가 관객들의 시선을 강탈해 뜨거운 호응을 얻는다고. 그런가 하면 ‘최애’ 멤버들의 공연을 관람한 장윤정은 일취월장한 실력에 “너네 진짜 너무 늘었어. 그냥 해주는 말이 아냐”라며 폭풍 칭찬을 쏟아낸다. 이어 김신영과 이특은 “오디션 볼 때보다 안 떨었어”, “진짜로”라고 해 공연에 대한 기대감을 한껏 끌어올리고 있다. 한편 세계화를 목표로 하는 ‘최애’ 그룹을 위해 역대급 관객을 초대, 더욱 긴장감 넘치는 공연이 펼쳐진다. 여기에 관객들은 최애와 차애를 투표, 어떤 멤버가 관객들의 마음을 사로잡을지 호기심을 자아낸다. 과연 ‘최애’ 멤버들은 부족한 연습 기간에도 팀워크, 안무, 라이브까지 소화해야 하는 어려운 첫 미션을 완성도 높은 무대로 보여줄 수 있을지, 최종 멤버로 발탁되기 전보다 실력이 얼마나 향상됐을지 본 방송을 향한 기대가 증폭된다.  최애 엔터테인먼트


In [35]:
print(content)

 ‘최애 트로트 그룹’의 첫 공연에 예상치 못한 관객들이 초대돼 귀추가 주목된다. 오늘(1일) 오후 9시 50분 방송되는 MBC 예능프로그램 ‘최애 엔터테인먼트’(연출 오누리, 이민지) 5회에서는 ‘최애’ 멤버들이 우여곡절 끝에 준비한 첫 공연을 시작, 갈고 닦은 실력을 뽐낸다. 이날 방송에서는 ‘최애’ 멤버들의 공연 장소에 도착, 위압감 느껴지는 야외무대에 일동 긴장하는 모습을 보인다. MJ와 추혁진은 “안전한 곳 맞죠?”, “약간 무서운데”라며 불안한 눈빛을 보이는가 하면, 옥진욱은 “이런 걸 또 언제 해보겠어”라며 막내의 긍정적인 에너지를 발산하기도. 과연 이들을 이토록 떨게 한 무대는 어떤 모습일지 궁금증을 더하고 있다. 또한 첫 무대에 오른 ‘최애’ 멤버들이 오색찬란한 트로트 가수 의상을 맞춰 입어 눈길을 끈다. 살짝만 움직여도 빤짝이는 셔츠핏과 부쩍 물오른 외모가 관객들의 시선을 강탈해 뜨거운 호응을 얻는다고. 그런가 하면 ‘최애’ 멤버들의 공연을 관람한 장윤정은 일취월장한 실력에 “너네 진짜 너무 늘었어. 그냥 해주는 말이 아냐”라며 폭풍 칭찬을 쏟아낸다. 이어 김신영과 이특은 “오디션 볼 때보다 안 떨었어”, “진짜로”라고 해 공연에 대한 기대감을 한껏 끌어올리고 있다. 한편 세계화를 목표로 하는 ‘최애’ 그룹을 위해 역대급 관객을 초대, 더욱 긴장감 넘치는 공연이 펼쳐진다. 여기에 관객들은 최애와 차애를 투표, 어떤 멤버가 관객들의 마음을 사로잡을지 호기심을 자아낸다. 과연 ‘최애’ 멤버들은 부족한 연습 기간에도 팀워크, 안무, 라이브까지 소화해야 하는 어려운 첫 미션을 완성도 높은 무대로 보여줄 수 있을지, 최종 멤버로 발탁되기 전보다 실력이 얼마나 향상됐을지 본 방송을 향한 기대가 증폭된다.  최애 엔터테인먼트


In [ ]:
news